1. Import all the relevant libraries
2. Drop all the unnecessary attributes
3. Create all the necessary attributes
4. How?
    a. We have rows corresponding to a particular traffic stop, with attributes of race, date, time, etc.
    b. Make a new dataset which has the attributes date, number of white subjects, number of african-american subjects
    c. From now on, we do everything in this new dataset
    d. Total population = 689,000 ; White population = 689000 x 0.56 = 385,000 ; African-American Population = 172,250
    e. Stops per 1000 white people = (Number of white stops/white population)x1000
    f. Stops per 1000 African-American people = (Number of African American stops/Aferican-American Population)x1000
    g. These two are new attributes which we will use
    h. Assume stops per 1000 white people and stops per 1000 african americans to be two normal populations with unknown means and variances.
    i. Assuming the variances to be equal, find the 95% confidence interval for the difference in means of the two populations.


In [4]:
# import all the relevant libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import tabulate as tb


In [5]:
# imprting the dataset tn_nashville_2020_04_01.csv
dataset = pd.read_csv('./Datasets/tn_nashville_2020_04_01.csv')


C:\Users\USER\AppData\Local\Temp\ipykernel_14200\3293555202.py:2: DtypeWarning: Columns (4,6,11,12,13,15,16,17,18,21) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv('./Datasets/tn_nashville_2020_04_01.csv')


In [6]:
# Dropping all columns except date, subject_race
dataset =  dataset[['date','subject_race']].copy()
# Displaying the head
print("Head of the dataset:")
display(dataset.head())

# Displaying the tail
print("\nTail of the dataset:")
display(dataset.tail())

Head of the dataset:


,date,subject_race
0,10-10-2010,black
1,10-10-2010,white
2,10-10-2010,white
3,10-10-2010,white
4,10-10-2010,black



Tail of the dataset:


,date,subject_race
1048570,16-10-2017,white
1048571,16-10-2017,white
1048572,16-10-2017,hispanic
1048573,16-10-2017,white
1048574,16-10-2017,white


In [7]:
# making a new dataset with date, number of traffic stops for white people, number of traffic stops for african-americans on that date

# Ensure the 'date' and 'subject_race' columns exist and have no missing values
dataset = dataset.dropna(subset=['date', 'subject_race'])

# Convert the 'date' column to datetime format
dataset['date'] = pd.to_datetime(dataset['date'], errors='coerce')

# Drop rows with invalid dates
dataset = dataset.dropna(subset=['date'])

# Filter the dataset by race and count the number of stops for each race per day
white_stops = dataset[dataset['subject_race'] == 'white'].groupby('date').size().reset_index(name='White_Stops')
black_stops = dataset[dataset['subject_race'] == 'black'].groupby('date').size().reset_index(name='Black_Stops')

# Merge the two datasets on the 'date' column
stops_by_date = pd.merge(white_stops, black_stops, on='date', how='outer').fillna(0)

# Display the resulting dataset
print("New dataset with traffic stops by date:")
print(stops_by_date.head())

New dataset with traffic stops by date:
        date  White_Stops  Black_Stops
0 2010-01-01          148          137
1 2010-01-10          610          349
2 2010-01-11          749          489
3 2010-01-12          676          451
4 2010-02-01          359          263


In [8]:
# Total population = 689,000 ; White population = 689000 x 0.56 = 385,000 ; African-American Population = 172,250
    # e. Stops per 10000 white people = (Number of white stops/white population)x1000
    # f. Stops per 10000 African-American people = (Number of African American stops/Aferican-American Population)x1000
    # g. These two are new attributes which we will use

# Define the population sizes
total_population = 689000
white_population = total_population * 0.56  # 385,000
black_population = total_population * 0.25  # 172,250

# Calculate stops per 10000 people for each group
stops_by_date['Stops_per_10000_White'] = (stops_by_date['White_Stops'] / white_population) * 10000
stops_by_date['Stops_per_10000_Black'] = (stops_by_date['Black_Stops'] / black_population) * 10000

# Convert the 'date' column to string format with only the date part
stops_by_date['date'] = stops_by_date['date'].astype(str)

# Display the dataset using tabulate
print("Dataset with stops per 10000 people (Head):")
print(tb.tabulate(stops_by_date.head(), headers='keys', tablefmt='pretty'))

print("\nDataset with stops per 10000 people (Tail):")
print(tb.tabulate(stops_by_date.tail(), headers='keys', tablefmt='pretty'))

Dataset with stops per 10000 people (Head):
+---+------------+-------------+-------------+-----------------------+-----------------------+
|   |    date    | White_Stops | Black_Stops | Stops_per_10000_White | Stops_per_10000_Black |
+---+------------+-------------+-------------+-----------------------+-----------------------+
| 0 | 2010-01-01 |     148     |     137     |  3.8357868546547786   |   7.95355587808418    |
| 1 | 2010-01-10 |     610     |     349     |  15.809662036077128   |  20.261248185776488   |
| 2 | 2010-01-11 |     749     |     489     |   19.4121915820029    |  28.388969521044995   |
| 3 | 2010-01-12 |     676     |     451     |  17.520215633423177   |   26.18287373004354   |
| 4 | 2010-02-01 |     359     |     263     |   9.304374870412605   |  15.268505079825834   |
+---+------------+-------------+-------------+-----------------------+-----------------------+

Dataset with stops per 10000 people (Tail):
+-----+------------+-------------+-------------+--------

In [9]:
# Assume stops per 10000 white people and stops per 10000 African Americans to be two normal populations with unknown means and variances.
# i. Assuming the variances to be equal, find the 95% confidence interval for the difference in means of the two populations.

# Step 1: Calculate the means and variances of the two populations
mean_white = stops_by_date['Stops_per_10000_White'].mean()
mean_black = stops_by_date['Stops_per_10000_Black'].mean()
var_white = stops_by_date['Stops_per_10000_White'].var(ddof=1)  # Sample variance
var_black = stops_by_date['Stops_per_10000_Black'].var(ddof=1)  # Sample variance

# Step 2: Calculate the pooled variance
n_white = len(stops_by_date['Stops_per_10000_White'])
n_black = len(stops_by_date['Stops_per_10000_Black'])
pooled_variance = ((n_white - 1) * var_white + (n_black - 1) * var_black) / (n_white + n_black - 2)

# Step 3: Calculate the standard error of the difference in means
se_diff = (pooled_variance * (1 / n_white + 1 / n_black)) ** 0.5

# Step 4: Calculate the critical t-value for a 95% confidence interval
from scipy.stats import t
alpha = 0.05
df = n_white + n_black - 2
t_critical = t.ppf(1 - alpha / 2, df)

# Step 5: Calculate the margin of error
margin_of_error = t_critical * se_diff

# Step 6: Calculate the confidence interval
difference_in_means = mean_white - mean_black
lower_bound = difference_in_means - margin_of_error
upper_bound = difference_in_means + margin_of_error

# Display the results
print(f"Mean Stops per 10000 White: {mean_white}")
print(f"Mean Stops per 10000 Black: {mean_black}")
print(f"Pooled Variance: {pooled_variance}")
print(f"Standard Error of Difference: {se_diff}")
print(f"Degrees of Freedom: {df}")
print(f"Critical t-value: {t_critical}")
print(f"Margin of Error: {margin_of_error}")
print(f"95% Confidence Interval for the Difference in Means: ({lower_bound}, {upper_bound})")

Mean Stops per 10000 White: 13.21046856982805
Mean Stops per 10000 Black: 20.349094797952795
Pooled Variance: 46.39728319368809
Standard Error of Difference: 0.4511062581173831
Degrees of Freedom: 910
Critical t-value: 1.9625742877989996
Margin of Error: 0.8853295432463948
95% Confidence Interval for the Difference in Means: (-8.023955771371138, -6.253296684878349)
